# Assignment 3
## Allan Robey
## 22111007

In [1]:
# import the required modules ....
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import ipywidgets as widgets
from vtk import *

In [2]:
# Load the dataset.....
reader = vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update()
data = reader.GetOutput()
num_dimensions = data.GetDimensions()
print(num_dimensions)
num_points = num_dimensions[0] * num_dimensions[1] * num_dimensions[2]
array_of_data = data.GetPointData().GetArray('ImageFile')
values = [array_of_data.GetTuple1(pid) for pid in range(num_points)]
coordinates = [data.GetPoint(pid) for pid in range(num_points)]
img_range = data.GetScalarRange()
# getting the x , y and z co-ordinates...
x = [i[0] for i in coordinates]
y = [i[1] for i in coordinates]
z = [i[2] for i in coordinates]

(75, 75, 75)


In [8]:
# Initial ISosurface
fig1 = go.FigureWidget(data=go.Isosurface(
    x=x,
    y=y,
    z=z,
    value=values,
    isomin=0.0,
    isomax=0.0,
    cauto = False,
    cmin = min(values),
    cmax = max(values),
    caps=dict(x_show=False, y_show=False),
    colorscale='plasma',
    opacity = 1,
    colorbar = dict(thickness=20, ticklen=4),
    showscale = False
    
))

In [9]:
# Initial Histogram
# Create a histogram trace
histogram_trace = go.Histogram(x=values, nbinsx=30)

# Create a layout object
layout = go.Layout(
    #title='Histogram',
    xaxis=dict(title='Vortex Scalar Values'),
    yaxis=dict(title='Frequency'),
)

# Create a figure object
fig2 = go.FigureWidget(data=[histogram_trace], layout=layout)

In [10]:
# Create the widgets with slider and button
slider = widgets.FloatSlider(value=0.0, min=img_range[0], max=img_range[1], description='Isovalue',continuous_update=False,step=0.01)
button = widgets.Button(description='Reset')


# method to update the graph
def update_graph(change):
    isovalue = change.new
    fig1.data[0].isomin = isovalue
    fig1.data[0].isomax = isovalue
    # Update histogram data based on slider value
    filtered_values = [val for val in values if val >= isovalue - 0.25 and val <= isovalue + 0.25]
    fig2.data[0].x = filtered_values
    

# method to reset the graph    
def reset_graph(button):
    # Reset isovalue to initial value
    slider.value = 0.0
    isovalue = 0.0
    # Reset the isosurface value
    fig1.data[0].update(isomin=isovalue, isomax=isovalue)
    # Reset histogram data
    fig2.data[0].x = values
    
# Link the slider to the update function
slider.observe(update_graph, names='value')

# Link the button to the reset function
button.on_click(reset_graph)

# Create the layout for the widget
graph_widget = widgets.VBox([
    widgets.HBox([slider, button]), 
    widgets.HBox([fig1, fig2])
])

# Display the widget
display(graph_widget)

##### 